# 🚀 Comprehensive Multi-Page Performance Analysis

## Advanced Core Web Vitals Analysis Across Multiple Pages

This notebook analyzes lighthouse-generated Core Web Vitals data for **3 frameworks** (Next.js, Nuxt.js, SvelteKit) across **4 rendering strategies** (CSR, SSR, SSG, ISR) on **3 different pages** (Blog, About, BlogPost).

### 📊 Analysis Structure:
- **Individual Page Analysis** - Detailed performance metrics for each page
- **Cross-Page Comparison** - Compare performance patterns across different page types
- **Structured Output** - Results saved in organized folders for each page
- **Comprehensive Insights** - Best practices and recommendations per page type

### 🎯 Key Metrics Analyzed:
- **First Contentful Paint (FCP)** - Time until first text/image appears
- **Largest Contentful Paint (LCP)** - Time until largest content element loads
- **Speed Index** - How quickly content is visually displayed
- **Time to Interactive (TTI)** - Time until page becomes fully interactive
- **Total Blocking Time (TBT)** - Time when main thread is blocked
- **Cumulative Layout Shift (CLS)** - Visual stability metric

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from IPython.display import display, HTML
import warnings
import os
import json
from pathlib import Path
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure plotly for better display
import plotly.io as pio
pio.renderers.default = "notebook"

print("✅ All libraries imported successfully!")
print("📁 Setting up directory structure...")

# Create output directory structure
output_dir = Path('output')
pages = ['blog', 'about', 'blogPost']

for page in pages:
    page_dir = output_dir / page
    page_dir.mkdir(parents=True, exist_ok=True)
    
# Create comparison directory
comparison_dir = output_dir / 'cross_page_comparison'
comparison_dir.mkdir(parents=True, exist_ok=True)

print("✅ Directory structure created!")

In [ ]:
# Load all data files
input_files = {
    'blog': 'inputs/master_blog_comparison_slow4g_2025-06-10T09-41-32-118Z.csv',
    'about': 'inputs/master_about_comparison_slow4g_2025-06-10T10-11-32-511Z.csv',
    'blogPost': 'inputs/master_blogPost_comparison_slow4g_2025-06-10T10-38-29-481Z.csv'
}

# Load all datasets
datasets = {}
for page_name, file_path in input_files.items():
    try:
        df = pd.read_csv(file_path)
        df['Page_Type'] = page_name  # Add page identifier
        datasets[page_name] = df
        print(f"✅ {page_name.upper()} data loaded: {df.shape}")
    except Exception as e:
        print(f"❌ Error loading {page_name}: {e}")

print(f"\n📊 Total pages loaded: {len(datasets)}")

# Display sample from each dataset
for page_name, df in datasets.items():
    print(f"\n🔍 {page_name.upper()} Sample:")
    print(f"Frameworks: {df['Framework'].unique()}")
    print(f"Strategies: {df['Strategy'].unique()}")
    print(f"Total runs per test: {df['Total_Runs'].iloc[0]}")

## 🔧 Data Processing & Preparation

In [ ]:
# Function to clean and structure data
def process_page_data(df, page_name):
    """Process raw lighthouse data into structured format"""
    metrics_data = []
    
    for _, row in df.iterrows():
        base_info = {
            'Page_Type': page_name,
            'App_Name': row['App_Name'],
            'Framework': row['Framework'],
            'Strategy': row['Strategy'],
            'Total_Runs': row['Total_Runs']
        }
        
        # Core Web Vitals metrics
        metrics = {
            'FCP': {'value': row['First Contentful Paint_Avg_Value'], 'score': row['First Contentful Paint_Avg_Score_%']},
            'LCP': {'value': row['Largest Contentful Paint_Avg_Value'], 'score': row['Largest Contentful Paint_Avg_Score_%']},
            'SI': {'value': row['Speed Index_Avg_Value'], 'score': row['Speed Index_Avg_Score_%']},
            'TTI': {'value': row['Interactive_Avg_Value'], 'score': row['Interactive_Avg_Score_%']},
            'TBT': {'value': row['Total Blocking Time_Avg_Value'], 'score': row['Total Blocking Time_Avg_Score_%']},
            'CLS': {'value': row['Cumulative Layout Shift_Avg_Value'], 'score': row['Cumulative Layout Shift_Avg_Score_%']}
        }
        
        for metric_name, metric_data in metrics.items():
            metrics_data.append({
                **base_info,
                'Metric': metric_name,
                'Value': metric_data['value'],
                'Score': metric_data['score']
            })
    
    return pd.DataFrame(metrics_data)

# Process all datasets
cleaned_datasets = {}
for page_name, df in datasets.items():
    cleaned_df = process_page_data(df, page_name)
    cleaned_datasets[page_name] = cleaned_df
    print(f"✅ {page_name.upper()} processed: {cleaned_df.shape}")

# Combine all data for cross-page analysis
all_data = pd.concat(cleaned_datasets.values(), ignore_index=True)
print(f"\n🔗 Combined dataset shape: {all_data.shape}")

# Display sample of combined data
print("\n📋 Combined Data Sample:")
display(all_data.head())

## 📊 Individual Page Analysis

In [ ]:
# Function to analyze individual page performance
def analyze_page_performance(df, page_name):
    """Comprehensive analysis for a single page"""
    print(f"\n🔍 ANALYZING {page_name.upper()} PAGE PERFORMANCE")
    print("=" * 50)
    
    # Create pivot tables
    values_pivot = df.pivot_table(
        index=['Framework', 'Strategy'], 
        columns='Metric', 
        values='Value', 
        aggfunc='mean'
    ).round(3)
    
    scores_pivot = df.pivot_table(
        index=['Framework', 'Strategy'], 
        columns='Metric', 
        values='Score', 
        aggfunc='mean'
    ).round(1)
    
    # Calculate rankings
    framework_rankings = df.groupby('Framework')['Score'].mean().sort_values(ascending=False)
    strategy_rankings = df.groupby('Strategy')['Score'].mean().sort_values(ascending=False)
    combination_rankings = df.groupby(['Framework', 'Strategy'])['Score'].mean().sort_values(ascending=False)
    
    # Best performers by metric
    metric_leaders = {}
    metrics_list = ['FCP', 'LCP', 'SI', 'TTI', 'TBT', 'CLS']
    for metric in metrics_list:
        metric_data = df[df['Metric'] == metric]
        best = metric_data.loc[metric_data['Score'].idxmax()]
        metric_leaders[metric] = {
            'framework': best['Framework'],
            'strategy': best['Strategy'],
            'score': best['Score'],
            'value': best['Value']
        }
    
    # Print results
    print(f"\n🏆 TOP PERFORMERS:")
    print(f"Best Framework: {framework_rankings.index[0]} ({framework_rankings.iloc[0]:.1f}%)")
    print(f"Best Strategy: {strategy_rankings.index[0]} ({strategy_rankings.iloc[0]:.1f}%)")
    print(f"Best Combination: {combination_rankings.index[0][0]} + {combination_rankings.index[0][1]} ({combination_rankings.iloc[0]:.1f}%)")
    
    print(f"\n🎯 METRIC LEADERS:")
    for metric, leader in metric_leaders.items():
        print(f"{metric}: {leader['framework']} + {leader['strategy']} ({leader['score']:.1f}%)")
    
    return {
        'values_pivot': values_pivot,
        'scores_pivot': scores_pivot,
        'framework_rankings': framework_rankings,
        'strategy_rankings': strategy_rankings,
        'combination_rankings': combination_rankings,
        'metric_leaders': metric_leaders
    }

# Analyze each page
page_analyses = {}
for page_name, df in cleaned_datasets.items():
    analysis = analyze_page_performance(df, page_name)
    page_analyses[page_name] = analysis

In [ ]:
# Create visualizations for each page
def create_page_visualizations(df, page_name, analysis_results):
    """Create comprehensive visualizations for a single page"""
    
    # 1. Performance Heatmap
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Scores heatmap
    sns.heatmap(analysis_results['scores_pivot'], annot=True, cmap='RdYlGn', center=75, 
               fmt='.1f', ax=ax1, cbar_kws={'label': 'Score (%)'}, linewidths=0.5)
    ax1.set_title(f'🎯 {page_name.upper()} Performance Scores\n(Higher is Better)', 
                 fontsize=14, fontweight='bold')
    ax1.set_xlabel('Core Web Vitals Metrics', fontweight='bold')
    ax1.set_ylabel('Framework + Strategy', fontweight='bold')
    
    # Values heatmap
    sns.heatmap(analysis_results['values_pivot'], annot=True, cmap='RdYlGn_r', 
               fmt='.2f', ax=ax2, cbar_kws={'label': 'Value (seconds/units)'}, linewidths=0.5)
    ax2.set_title(f'⏱️ {page_name.upper()} Performance Values\n(Lower is Generally Better)', 
                 fontsize=14, fontweight='bold')
    ax2.set_xlabel('Core Web Vitals Metrics', fontweight='bold')
    ax2.set_ylabel('Framework + Strategy', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'output/{page_name}/{page_name}_heatmap.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 2. Framework Performance Bar & Line Chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # Overall Framework Performance (Bar Chart)
    framework_scores = df.groupby('Framework')['Score'].mean().sort_values(ascending=False)
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    bars = ax1.bar(framework_scores.index, framework_scores.values, 
                  color=colors[:len(framework_scores)], alpha=0.8, edgecolor='black', linewidth=1.2)
    
    ax1.set_title(f'🏆 {page_name.upper()} Overall Framework Performance', 
                 fontsize=14, fontweight='bold')
    ax1.set_ylabel('Average Performance Score (%)', fontweight='bold')
    ax1.set_ylim(0, 100)
    ax1.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add value labels on bars
    for bar, score in zip(bars, framework_scores.values):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{score:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)
    
    # Framework Performance by Metric (Line Chart)
    metrics_list = ['FCP', 'LCP', 'SI', 'TTI', 'TBT', 'CLS']
    framework_metric_data = []
    
    for framework in df['Framework'].unique():
        fw_data = df[df['Framework'] == framework]
        metric_scores = []
        for metric in metrics_list:
            metric_score = fw_data[fw_data['Metric'] == metric]['Score'].mean()
            metric_scores.append(metric_score)
        framework_metric_data.append((framework, metric_scores))
    
    # Plot lines for each framework
    line_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
    markers = ['o', 's', '^']
    
    for i, (framework, scores) in enumerate(framework_metric_data):
        ax2.plot(metrics_list, scores, 
                color=line_colors[i % len(line_colors)], 
                marker=markers[i % len(markers)], 
                linewidth=3, markersize=8, 
                label=framework, alpha=0.8)
        
        # Add value labels on points
        for j, score in enumerate(scores):
            ax2.annotate(f'{score:.0f}', 
                       (j, score), 
                       textcoords="offset points", 
                       xytext=(0,10), 
                       ha='center', fontsize=9, fontweight='bold')
    
    ax2.set_title(f'📈 {page_name.upper()} Performance by Metric', 
                 fontsize=14, fontweight='bold')
    ax2.set_ylabel('Performance Score (%)', fontweight='bold')
    ax2.set_xlabel('Core Web Vitals Metrics', fontweight='bold')
    ax2.set_ylim(0, 105)
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.legend(title='Framework', loc='lower right', fontsize=10)
    
    plt.suptitle(f'🚀 {page_name.upper()} Framework Performance Analysis', 
                fontsize=18, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.savefig(f'output/{page_name}/{page_name}_framework_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 3. Interactive Radar Chart
    fig = go.Figure()
    
    frameworks = analysis_results['scores_pivot'].index.get_level_values('Framework').unique()
    strategies = analysis_results['scores_pivot'].index.get_level_values('Strategy').unique()
    
    strategy_colors = {
        'Client-Side Rendering': '#FF6B6B',
        'Server-Side Rendering': '#4ECDC4', 
        'Static Site Generation': '#45B7D1',
        'Incremental Static Regeneration': '#96CEB4'
    }
    
    for framework in frameworks:
        for strategy in strategies:
            if (framework, strategy) in analysis_results['scores_pivot'].index:
                values = analysis_results['scores_pivot'].loc[(framework, strategy)].tolist()
                values.append(values[0])  # Close the radar chart
                
                fig.add_trace(go.Scatterpolar(
                    r=values,
                    theta=metrics_list + [metrics_list[0]],
                    fill='toself',
                    name=f'{framework} - {strategy}',
                    line_color=strategy_colors.get(strategy, '#666666'),
                    opacity=0.7
                ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 100], tickfont=dict(size=10)),
            angularaxis=dict(tickfont=dict(size=12))
        ),
        showlegend=True,
        title={
            'text': f'🎯 {page_name.upper()} Performance Radar Chart<br><sub>Core Web Vitals Scores (0-100%)</sub>',
            'x': 0.5,
            'font': {'size': 16}
        },
        height=700
    )
    
    fig.write_html(f'output/{page_name}/{page_name}_radar_chart.html')
    fig.show()

# Create visualizations for each page
for page_name, df in cleaned_datasets.items():
    print(f"\n📊 Creating visualizations for {page_name.upper()}...")
    create_page_visualizations(df, page_name, page_analyses[page_name])
    print(f"✅ {page_name.upper()} visualizations saved!")

## 🔄 Cross-Page Performance Comparison

In [ ]:
# Cross-page analysis
print("🔄 CROSS-PAGE PERFORMANCE ANALYSIS")
print("=" * 40)

# Overall page performance comparison
page_performance = all_data.groupby('Page_Type')['Score'].agg(['mean', 'std', 'min', 'max']).round(2)
page_performance.columns = ['Average_Score', 'Std_Dev', 'Min_Score', 'Max_Score']
page_performance = page_performance.sort_values('Average_Score', ascending=False)

print("\n📊 OVERALL PAGE PERFORMANCE RANKING:")
for i, (page, stats) in enumerate(page_performance.iterrows(), 1):
    print(f"{i}. {page.upper()}: {stats['Average_Score']:.1f}% (±{stats['Std_Dev']:.1f})")

display(page_performance)

# Framework performance across pages
framework_across_pages = all_data.groupby(['Page_Type', 'Framework'])['Score'].mean().unstack().round(1)
print("\n🏗️ FRAMEWORK PERFORMANCE ACROSS PAGES:")
display(framework_across_pages)

# Strategy performance across pages
strategy_across_pages = all_data.groupby(['Page_Type', 'Strategy'])['Score'].mean().unstack().round(1)
print("\n🔄 STRATEGY PERFORMANCE ACROSS PAGES:")
display(strategy_across_pages)

# Metric performance across pages
metric_across_pages = all_data.groupby(['Page_Type', 'Metric'])['Score'].mean().unstack().round(1)
print("\n🎯 METRIC PERFORMANCE ACROSS PAGES:")
display(metric_across_pages)

In [ ]:
# Create cross-page visualizations

# 1. Page Performance Comparison
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 12))

# Overall page scores
page_scores = all_data.groupby('Page_Type')['Score'].mean().sort_values(ascending=False)
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
bars = ax1.bar(page_scores.index, page_scores.values, color=colors, alpha=0.8)
ax1.set_title('📊 Overall Page Performance', fontsize=14, fontweight='bold')
ax1.set_ylabel('Average Score (%)')
ax1.set_ylim(0, 100)
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bar, score in zip(bars, page_scores.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
            f'{score:.1f}%', ha='center', fontweight='bold')

# Framework performance across pages
framework_across_pages.plot(kind='bar', ax=ax2, color=['#FF9999', '#66B2FF', '#99FF99'])
ax2.set_title('🏗️ Framework Performance Across Pages', fontsize=14, fontweight='bold')
ax2.set_ylabel('Average Score (%)')
ax2.legend(title='Framework', bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(axis='y', alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# Strategy performance across pages
strategy_across_pages.plot(kind='bar', ax=ax3)
ax3.set_title('🔄 Strategy Performance Across Pages', fontsize=14, fontweight='bold')
ax3.set_ylabel('Average Score (%)')
ax3.legend(title='Strategy', bbox_to_anchor=(1.05, 1), loc='upper left')
ax3.grid(axis='y', alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# Page consistency (standard deviation)
page_consistency = all_data.groupby('Page_Type')['Score'].std().sort_values()
bars = ax4.bar(page_consistency.index, page_consistency.values, color=colors, alpha=0.8)
ax4.set_title('📈 Page Performance Consistency\n(Lower = More Consistent)', fontsize=14, fontweight='bold')
ax4.set_ylabel('Standard Deviation')
ax4.grid(axis='y', alpha=0.3)

# Add value labels
for bar, std in zip(bars, page_consistency.values):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
            f'{std:.1f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('output/cross_page_comparison/cross_page_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# 2. Detailed Cross-Page Heatmap
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Framework heatmap
sns.heatmap(framework_across_pages, annot=True, cmap='RdYlGn', center=75, 
           fmt='.1f', ax=axes[0], cbar_kws={'label': 'Score (%)'}, linewidths=0.5)
axes[0].set_title('🏗️ Framework Performance\nAcross Pages', fontweight='bold')
axes[0].set_xlabel('Framework')
axes[0].set_ylabel('Page Type')

# Strategy heatmap
sns.heatmap(strategy_across_pages, annot=True, cmap='RdYlGn', center=75, 
           fmt='.1f', ax=axes[1], cbar_kws={'label': 'Score (%)'}, linewidths=0.5)
axes[1].set_title('🔄 Strategy Performance\nAcross Pages', fontweight='bold')
axes[1].set_xlabel('Strategy')
axes[1].set_ylabel('Page Type')

# Metric heatmap
sns.heatmap(metric_across_pages, annot=True, cmap='RdYlGn', center=75, 
           fmt='.1f', ax=axes[2], cbar_kws={'label': 'Score (%)'}, linewidths=0.5)
axes[2].set_title('🎯 Metric Performance\nAcross Pages', fontweight='bold')
axes[2].set_xlabel('Metric')
axes[2].set_ylabel('Page Type')

plt.tight_layout()
plt.savefig('output/cross_page_comparison/cross_page_heatmaps.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Interactive cross-page comparison chart
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Overall Page Performance', 'Framework Consistency Across Pages',
                   'Strategy Effectiveness by Page', 'Metric Performance Distribution'),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "bar"}, {"type": "box"}]]
)

# 1. Overall page performance
page_scores = all_data.groupby('Page_Type')['Score'].mean().sort_values(ascending=False)
fig.add_trace(
    go.Bar(x=page_scores.index, y=page_scores.values, 
           name='Page Performance', marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1']),
    row=1, col=1
)

# 2. Framework consistency scatter
for framework in all_data['Framework'].unique():
    fw_data = all_data[all_data['Framework'] == framework]
    page_means = fw_data.groupby('Page_Type')['Score'].mean()
    page_stds = fw_data.groupby('Page_Type')['Score'].std()
    
    fig.add_trace(
        go.Scatter(x=page_means.values, y=page_stds.values,
                  mode='markers+text', name=framework,
                  text=page_means.index, textposition='top center',
                  marker=dict(size=12, opacity=0.7)),
        row=1, col=2
    )

# 3. Strategy effectiveness
strategy_page_scores = all_data.groupby(['Strategy', 'Page_Type'])['Score'].mean().unstack()
for strategy in strategy_page_scores.index:
    fig.add_trace(
        go.Bar(x=strategy_page_scores.columns, y=strategy_page_scores.loc[strategy],
               name=strategy),
        row=2, col=1
    )

# 4. Metric distribution
for page in all_data['Page_Type'].unique():
    page_data = all_data[all_data['Page_Type'] == page]
    fig.add_trace(
        go.Box(y=page_data['Score'], name=page),
        row=2, col=2
    )

# Update layout
fig.update_layout(
    title='🔄 Comprehensive Cross-Page Performance Analysis',
    height=800,
    showlegend=True
)

fig.update_xaxes(title_text="Page Type", row=1, col=1)
fig.update_yaxes(title_text="Average Score (%)", row=1, col=1)
fig.update_xaxes(title_text="Mean Score", row=1, col=2)
fig.update_yaxes(title_text="Standard Deviation", row=1, col=2)
fig.update_xaxes(title_text="Page Type", row=2, col=1)
fig.update_yaxes(title_text="Score (%)", row=2, col=1)
fig.update_yaxes(title_text="Score (%)", row=2, col=2)

fig.write_html('output/cross_page_comparison/interactive_cross_page_analysis.html')
fig.show()

## 💾 Export Results & Generate Reports

In [ ]:
# Function to save individual page results
def save_page_results(page_name, analysis_results, df):
    """Save comprehensive results for each page"""
    output_path = Path(f'output/{page_name}')
    
    # Save pivot tables
    analysis_results['values_pivot'].to_csv(output_path / f'{page_name}_performance_values.csv')
    analysis_results['scores_pivot'].to_csv(output_path / f'{page_name}_performance_scores.csv')
    
    # Save rankings
    analysis_results['framework_rankings'].to_csv(output_path / f'{page_name}_framework_rankings.csv')
    analysis_results['strategy_rankings'].to_csv(output_path / f'{page_name}_strategy_rankings.csv')
    analysis_results['combination_rankings'].to_csv(output_path / f'{page_name}_combination_rankings.csv')
    
    # Save detailed report
    with open(output_path / f'{page_name}_performance_report.txt', 'w') as f:
        f.write(f"{page_name.upper()} PAGE PERFORMANCE REPORT\n")
        f.write("=" * 50 + "\n\n")
        
        f.write("FRAMEWORK RANKINGS\n")
        f.write("-" * 20 + "\n")
        for i, (framework, score) in enumerate(analysis_results['framework_rankings'].items(), 1):
            f.write(f"{i}. {framework}: {score:.1f}%\n")
        
        f.write("\nSTRATEGY RANKINGS\n")
        f.write("-" * 20 + "\n")
        for i, (strategy, score) in enumerate(analysis_results['strategy_rankings'].items(), 1):
            f.write(f"{i}. {strategy}: {score:.1f}%\n")
        
        f.write("\nBEST COMBINATIONS\n")
        f.write("-" * 20 + "\n")
        for i, ((framework, strategy), score) in enumerate(analysis_results['combination_rankings'].head(5).items(), 1):
            f.write(f"{i}. {framework} + {strategy}: {score:.1f}%\n")
        
        f.write("\nMETRIC LEADERS\n")
        f.write("-" * 20 + "\n")
        for metric, leader in analysis_results['metric_leaders'].items():
            f.write(f"{metric}: {leader['framework']} + {leader['strategy']} ({leader['score']:.1f}%, {leader['value']:.3f})\n")
    
    # Save raw cleaned data
    df.to_csv(output_path / f'{page_name}_cleaned_data.csv', index=False)
    
    print(f"✅ {page_name.upper()} results saved to output/{page_name}/")

# Save results for each page
for page_name in cleaned_datasets.keys():
    save_page_results(page_name, page_analyses[page_name], cleaned_datasets[page_name])

print("\n💾 SAVING CROSS-PAGE COMPARISON RESULTS...")

# Save cross-page analysis results
comparison_path = Path('output/cross_page_comparison')

# Save cross-page tables
page_performance.to_csv(comparison_path / 'overall_page_performance.csv')
framework_across_pages.to_csv(comparison_path / 'framework_across_pages.csv')
strategy_across_pages.to_csv(comparison_path / 'strategy_across_pages.csv')
metric_across_pages.to_csv(comparison_path / 'metric_across_pages.csv')

# Save complete dataset
all_data.to_csv(comparison_path / 'complete_dataset.csv', index=False)

print("✅ Cross-page comparison results saved!")

In [ ]:
# Generate comprehensive final report
print("📋 GENERATING COMPREHENSIVE FINAL REPORT...")

with open('output/COMPREHENSIVE_PERFORMANCE_REPORT.txt', 'w') as f:
    f.write("🚀 COMPREHENSIVE MULTI-PAGE PERFORMANCE ANALYSIS REPORT\n")
    f.write("=" * 70 + "\n\n")
    
    # Executive Summary
    f.write("📊 EXECUTIVE SUMMARY\n")
    f.write("-" * 20 + "\n")
    f.write(f"Pages Analyzed: {len(cleaned_datasets)}\n")
    f.write(f"Frameworks Tested: {', '.join(all_data['Framework'].unique())}\n")
    f.write(f"Strategies Tested: {len(all_data['Strategy'].unique())}\n")
    f.write(f"Total Test Combinations: {len(all_data.groupby(['Page_Type', 'Framework', 'Strategy']))}\n")
    f.write(f"Overall Average Score: {all_data['Score'].mean():.1f}%\n\n")
    
    # Page Rankings
    f.write("🏆 OVERALL PAGE PERFORMANCE RANKINGS\n")
    f.write("-" * 40 + "\n")
    for i, (page, stats) in enumerate(page_performance.iterrows(), 1):
        f.write(f"{i}. {page.upper()}: {stats['Average_Score']:.1f}% (±{stats['Std_Dev']:.1f})\n")
    f.write("\n")
    
    # Individual Page Analysis
    for page_name, analysis in page_analyses.items():
        f.write(f"📖 {page_name.upper()} PAGE ANALYSIS\n")
        f.write("-" * 30 + "\n")
        f.write(f"Best Framework: {analysis['framework_rankings'].index[0]} ({analysis['framework_rankings'].iloc[0]:.1f}%)\n")
        f.write(f"Best Strategy: {analysis['strategy_rankings'].index[0]} ({analysis['strategy_rankings'].iloc[0]:.1f}%)\n")
        f.write(f"Best Combination: {analysis['combination_rankings'].index[0][0]} + {analysis['combination_rankings'].index[0][1]} ({analysis['combination_rankings'].iloc[0]:.1f}%)\n")
        
        f.write("\nMetric Leaders:\n")
        for metric, leader in analysis['metric_leaders'].items():
            f.write(f"  • {metric}: {leader['framework']} + {leader['strategy']} ({leader['score']:.1f}%)\n")
        f.write("\n")
    
    # Cross-Page Insights
    f.write("🔄 CROSS-PAGE INSIGHTS\n")
    f.write("-" * 25 + "\n")
    
    # Best overall framework
    overall_best_framework = all_data.groupby('Framework')['Score'].mean().sort_values(ascending=False)
    f.write(f"Best Overall Framework: {overall_best_framework.index[0]} ({overall_best_framework.iloc[0]:.1f}% avg across all pages)\n")
    
    # Best overall strategy
    overall_best_strategy = all_data.groupby('Strategy')['Score'].mean().sort_values(ascending=False)
    f.write(f"Best Overall Strategy: {overall_best_strategy.index[0]} ({overall_best_strategy.iloc[0]:.1f}% avg across all pages)\n")
    
    # Most consistent performer
    framework_consistency = all_data.groupby('Framework')['Score'].std().sort_values()
    f.write(f"Most Consistent Framework: {framework_consistency.index[0]} (σ={framework_consistency.iloc[0]:.1f})\n")
    
    strategy_consistency = all_data.groupby('Strategy')['Score'].std().sort_values()
    f.write(f"Most Consistent Strategy: {strategy_consistency.index[0]} (σ={strategy_consistency.iloc[0]:.1f})\n\n")
    
    # Key Recommendations
    f.write("💡 KEY RECOMMENDATIONS\n")
    f.write("-" * 25 + "\n")
    f.write(f"1. FOR MAXIMUM PERFORMANCE: Use {overall_best_framework.index[0]} with {overall_best_strategy.index[0]}\n")
    f.write(f"2. FOR CONSISTENCY: Choose {framework_consistency.index[0]} framework with {strategy_consistency.index[0]} strategy\n")
    f.write(f"3. PAGE-SPECIFIC OPTIMIZATION: Focus on {page_performance.index[-1]} page (lowest performer)\n")
    f.write(f"4. METRIC PRIORITIES: Address TBT and LCP metrics across all pages\n")
    f.write(f"5. TESTING APPROACH: Validate performance gains with A/B testing\n\n")
    
    # Performance Patterns
    f.write("📈 PERFORMANCE PATTERNS\n")
    f.write("-" * 25 + "\n")
    
    # Check if SSG/SSR outperforms CSR
    csr_score = all_data[all_data['Strategy'] == 'Client-Side Rendering']['Score'].mean()
    ssg_score = all_data[all_data['Strategy'] == 'Static Site Generation']['Score'].mean()
    if ssg_score > csr_score:
        f.write(f"• Static rendering strategies outperform client-side rendering ({ssg_score:.1f}% vs {csr_score:.1f}%)\n")
    
    # Framework performance gap
    fw_gap = overall_best_framework.iloc[0] - overall_best_framework.iloc[-1]
    if fw_gap > 10:
        f.write(f"• Significant framework performance differences detected ({fw_gap:.1f}% gap)\n")
    else:
        f.write(f"• Framework performance is relatively similar (within {fw_gap:.1f}%)\n")
    
    # Page complexity insights
    page_complexity = page_performance['Std_Dev'].sort_values(ascending=False)
    f.write(f"• {page_complexity.index[0]} page shows highest performance variability (σ={page_complexity.iloc[0]:.1f})\n")
    f.write(f"• {page_complexity.index[-1]} page shows most consistent performance (σ={page_complexity.iloc[-1]:.1f})\n\n")
    
    f.write("="*70 + "\n")
    f.write("📁 GENERATED FILES:\n")
    f.write("Individual page results: output/[page_name]/\n")
    f.write("Cross-page comparison: output/cross_page_comparison/\n")
    f.write("Complete dataset: output/cross_page_comparison/complete_dataset.csv\n")
    f.write("Interactive charts: *.html files in respective folders\n")
    f.write("="*70 + "\n")

print("✅ Comprehensive report generated: COMPREHENSIVE_PERFORMANCE_REPORT.txt")

# Generate summary statistics
summary_stats = {
    'total_pages': len(cleaned_datasets),
    'total_frameworks': len(all_data['Framework'].unique()),
    'total_strategies': len(all_data['Strategy'].unique()),
    'total_combinations': len(all_data.groupby(['Framework', 'Strategy'])),
    'overall_average_score': all_data['Score'].mean(),
    'best_page': page_performance.index[0],
    'best_framework': overall_best_framework.index[0],
    'best_strategy': overall_best_strategy.index[0],
    'analysis_timestamp': pd.Timestamp.now().isoformat()
}

with open('output/analysis_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)

print("\n🎉 ANALYSIS COMPLETE!")
print("📁 All results saved to the output/ directory")
print("📊 Check individual page folders for detailed analysis")
print("🔄 Check cross_page_comparison/ for comparative insights")
print("📋 Read COMPREHENSIVE_PERFORMANCE_REPORT.txt for executive summary")